# 1. Importar tablas

In [1]:
/ 1.1. Importar tabla empleados
employees:get `:data/employees;

/show employees;

In [2]:
/ 1.2. Importar tabla remuneraciones
remunerations:get `:data/remunerations;

/show employees;

In [3]:
/ 1.3. Importar tabla de bases mínimas y máximas
minMaxBasis:("DDFFI";enlist csv) 0: `$":data/minMaxBasis.csv";

/show minMaxBasis

# 2. Variables globales

In [4]:
argname:`commonBase,
        `profBase,
        `hExtraFMBase,
        `hExtraBase,
        `irpfBase,
        `meiBase;

argrems:(`sb`cs`ge`se; 
         `sb`cs`hefm`her`ge`se;
         `hefm;
         `her;
         `sb`cs`hefm`her`ge`se`pns;
         `sb`cs`ge`se);

argdate:0Nm,
        0Nm,
        0Nm,
        0Nm,
        0Nm,
        2023.01m;

# 3. Funciones


In [5]:
getremsum:{[name;lsim;sd]:() xkey select baseType:name, sum amount     /(get) (rem)uneration (sum)
                                  by nif, month 
                                  from remunerations 
                                  where (max remunerationType=/:lsim), month>=sd;}

minmax:{[n;cg;mo;bt]                                                   /values between (min)imumm & (max)imum
    if[bt=`profBase; cg:4];
    first exec maxBase & n | minBase 
    from minMaxBasis
    where contrGroup= 4&cg, mo within(startDate;endDate)};

# 4. Generación de las tablas

In [6]:
contributionBasis: (raze getremsum'[argname;argrems;argdate]) 
                    lj `nif xkey select contrGroup, nif from employees;

update amount: minmax'[amount;contrGroup;month;baseType] 
  from `contributionBasis 
  where (max baseType=/:`commonBase`profBase`meiBase);

delete contrGroup 
  from `contributionBasis;

Si se quisiese ver de forma más visual de las bases de cotización se puede pivotar la tabla de la siguiente manera:

In [7]:
show 0^exec `commonBase`profBase`hExtraFMBase`hExtraBase`irpfBase`meiBase#(baseType!amount) by nif,month from contributionBasis;

nif       month  | commonBase profBase hExtraFMBase hExtraBase irpfBase meiBa..
-----------------| ----------------------------------------------------------..
00193031S 2021.07| 1050       1050     746.0766     746.0766   746.0766 0    ..
00193031S 2021.08| 2604.284   2604.284 2604.284     2604.284   2604.284 0    ..
00193031S 2021.09| 2608.907   2608.907 2608.907     2608.907   2608.907 0    ..
00193031S 2021.10| 2559.828   2614.682 2614.682     2614.682   2614.682 0    ..
00193031S 2022.07| 1166.7     1166.7   744.8285     744.8285   744.8285 0    ..
00193031S 2022.08| 2459.812   2459.812 2459.812     2459.812   2459.812 0    ..
00193031S 2022.09| 2689.987   2689.987 2689.987     2689.987   2689.987 0    ..
00193031S 2022.10| 2566.43    2566.43  2566.43      2566.43    2566.43  0    ..
00193031S 2023.07| 1260       1260     763.1126     763.1126   763.1126 1260 ..
00193031S 2023.08| 2595.259   2595.259 2595.259     2595.259   2595.259 2595...
00193031S 2023.09| 2600.467   2600.467 2

# 5. Guardar datos en csv

In [8]:
`:data/contributionBasis set contributionBasis;

In [9]:
count contributionBasis

351174
